In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents =[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [9]:
q = 'the course has already started, can i still enroll?'

In [10]:
index.fit(documents)

In [14]:
boost = {'question': 3.0, 'section':0.5}

results = index.search(query=q, filter_dict = {'course': 'data-engineering-zoomcamp'},
             boost_dict = boost, 
             num_results = 5
            
            )

In [15]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [17]:
from openai import OpenAI

In [19]:
import os

In [22]:
os.environ["OPENAI_API_KEY"] = "your_api_key"

In [23]:
api_key = os.environ.get('OPENAI_API_KEY')

In [25]:
client = OpenAI(api_key=api_key)

In [29]:
# response = client.chat.completions.create(model="gpt-4o", 
#                                           messages = [{"role":"user", "content":"is it too late to join the course"}])

In [27]:
response = client.chat.completions.create(model="gpt-4o", 
                                          messages = [{"role":"user", "content":q}])

In [28]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on several factors, including the institution's policies, the type of course, and how far along the course is. Here are some steps you can take:\n\n1. **Check the Course Policy:** Look at the official website or course catalog for any information regarding late enrollment policies. Some courses may have a grace period during which late enrollment is allowed.\n\n2. **Contact the Instructor:** Reach out directly to the instructor or course coordinator. Explain your situation and ask if it’s possible to join the class late. They may be willing to make an exception or offer guidance on catching up.\n\n3. **Consult with Academic Advising:** Academic advisors can often provide insights into whether late enrollment is possible and the implications it might have on your academic progress.\n\n4. **Review Course Materials:** If you do get permission to enroll late, make sure to review all the materials that have been cove

In [35]:
prompt_template="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [32]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

In [33]:
print(context)

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question:Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the 

In [38]:
prompt = prompt_template.format(question=q, context=context).strip()

In [39]:
response = client.chat.completions.create(model="gpt-4o", 
                                          messages = [{"role":"user", "content":prompt}])

In [40]:
response.choices[0].message.content

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework assignments, but be mindful of the deadlines for the final projects to avoid last-minute issues.'